In [ ]:
#importanción de pandas
import pandas as pd


In [ ]:
#lectura del e importación del csv como DataFrame
df = pd.read_csv("./input/GSAF5.csv")

In [ ]:
df.shape

In [ ]:
print(df.columns)

In [ ]:
#voy a renombrar columnas para facilitar el acceso
df.columns =['caseNumber', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal(Y/N)', 'time',
       'species', 'investigatorSource', 'pdf', 'hrefFormula', 'href',
       'caseNumber1', 'caseNumber2', 'originalOrder', 'Unnamed22',
       'Unnamed23']
df.head()

In [ ]:
#Compruebo columnas con valores nulos
null_cols = df.isnull().sum()
null_cols[null_cols > 0]
null_cols

In [ ]:
#Guardo un data Frame con las columnas que no tienen valores nulos, están limpias
dfNotNull = df[["caseNumber","date","year","type","pdf","caseNumber1","caseNumber2","originalOrder"]]

In [ ]:
'''En este punto tomo la decisión de dropear del Data Frame principal las columnas Unnamed22 y 23,
pues la totalidad de sus registros prácticamente son nulos'''
df = df.drop(columns = ["Unnamed22","Unnamed23"])

In [ ]:
'''la siguiente columnas con bastantes nulos es edad, esta vez lo que voy a hacer es meter en el dfNotNull una columna
 que se llame age(-1NoData) con los valores validos mas -1 para los valores inválidos'''
dfNotNull["age(-1NoData)"] = df['age'].fillna(-1)

In [ ]:
'''La siguiente columna con muchos nulos, más de la mitad es time, voy a hacer un display para decidir si me interesa
guardar esa info'''

pd.DataFrame(df['time'].value_counts())

In [ ]:
'''Voy a cambiar los nulos, merece la pena limpiar la columna'''
dfNotNull["timeDirty"] = df['time'].fillna("noData")

In [ ]:
'''Voy a analizar species'''
pd.DataFrame(df["species"].value_counts())

In [ ]:
'''Decido cambiar los nulos a Shark involvement not confirmed, parece lógico agrupar en esta categoría los nulos'''
dfNotNull["species"] = df['species'].fillna("Shark involvement not confirmed")


In [ ]:
'''el resto de columnas que tienen no válidos contienen pocos por lo que voy a meter valores que no lleven a confusión'''

In [ ]:
dfNotNull["country"] = df['country'].fillna("unknown")
dfNotNull["area"] = df['area'].fillna("unknown")
dfNotNull["location"] = df['location'].fillna("unknown")
dfNotNull["activity"] = df['activity'].fillna("unknown")
dfNotNull["name"] = df['name'].fillna("unknown")
dfNotNull["sex"] = df['sex'].fillna("unknown")
dfNotNull["injury"] = df['injury'].fillna("unknown")
dfNotNull["fatal(Y/N/unknow)"] = df['fatal(Y/N)'].fillna("unknown")
dfNotNull["investigatorSource"] = df['investigatorSource'].fillna("unknown")
dfNotNull["hrefFormula"] = df['hrefFormula'].fillna("unknown")
dfNotNull["href"] = df['href'].fillna("unknown")
#Compruebo que no hay nulos
null_cols = dfNotNull.isnull().sum()
#null_cols[null_cols > 0]
null_cols


In [ ]:
#limpieza de "fatal"
fatal = "fatal(Y/N/unknow)"
cleanFatal = dfNotNull[fatal]
cleanFatal.value_counts()

In [ ]:
from functions import cleanTextInFatal
cleanFatal = cleanFatal.apply(cleanTextInFatal)
cleanFatal.value_counts()


In [ ]:
#Lo añado a un dataFrame que tendrá los datos limpios
dfClean = pd.DataFrame(cleanFatal)
dfNotNull = dfNotNull.drop(columns =fatal)
#elimino los objetos que no voy a usar
del cleanFatal
del fatal

In [ ]:
dfNotNull.columns


In [ ]:
#voy a analizar type
dfNotNull["type"].value_counts()

In [ ]:
#diría que no tiene demasiado sentido Sea disaster, boat y boating, los voy a cambiar a junto con Ivalid a unknown
dfClean["type"]=dfNotNull["type"].replace(
    ['Invalid', 'Sea Disaster',"Boat","Boating"],
    'UNKNOWN',
    
)
dfNotNull=dfNotNull.drop(columns = "type")
dfClean["type"].value_counts()

In [ ]:
dfNotNull.columns

In [ ]:
#paso a activity
activitySerie = dfNotNull["activity"]
activitySerie.value_counts().head(40)


In [ ]:
"Voy a hacer alguno replace para unificar algunos datos fáciles"
activitySerie=activitySerie.replace(
    ['Swimming ',"Floating on his back"],
    'Swimming',
    
)
activitySerie=activitySerie.replace(
    'Fishing ',
    'Fishing',
    
)
activitySerie=activitySerie.replace(
    ['Diving for abalone',"Diving for trochus","Fishing for mackerel","Free diving for abalone","Sponge diving",
    "Hard hat diving", "Diving for coins","Freediving"
],
"Diving"    
)

activitySerie=activitySerie.replace(
    ['Surfing (sitting on his board)',"Sitting on surfboard"],
"Surfing"   
)

activitySerie=activitySerie.replace(
    ['Treading water',"Playing","Walking","Standing"],
"Wading"  
)
activitySerie=activitySerie.replace(
    ['Spearfishing ',"Skindiving","Spearfishing on Scuba","Scuba diving (but on surface)","Scuba diving (submerged)"],
"Spearfishing"   
)
activitySerie=activitySerie.replace(
    ["Murder","Sea Disaster"],
"unknown"  
)
activitySerie=activitySerie.replace(
    ['Fell overboard ',"Canoeing","Fell into the water","Boat","Dangling feet in the water","Sailing "],
"Boating"   
)

activitySerie=activitySerie.replace(
    ['Surf-skiing  ',"Surf skiing "],
"Surf skiing"   
)

activitySerie=activitySerie.replace(
    ['Sea Disaster '],
"Sea Disaster"   
)
 
activitySerie.value_counts().head(40)


In [ ]:
#Voy a categorizar la columna en los grupos que actualmente tienen más de 6 registros, 
#para ello hago una lista con los que tienen menos de 11 para ir filtrando y ver en qué categoria los meto 

badGroupsInActivity =pd.DataFrame(activitySerie.value_counts()).query("activity<11").index

badGroupsInActivity = [str(ele) for ele in badGroupsInActivity]
print(len(badGroupsInActivity))

In [ ]:
#Programé unas funciones para que me formatearan en las categorías la columna
from functions import cleanActivity


In [ ]:
activitySerie = activitySerie.apply(cleanActivity,lista = badGroupsInActivity)

activitySerie.value_counts().head(40)


In [ ]:
"Vuelvo a hacer print de la lista de valores que no quiero por si me hubiera dejado algo"
badGroupsInActivity =pd.DataFrame(activitySerie.value_counts()).query("activity<11").index

badGroupsInActivity = [str(ele) for ele in badGroupsInActivity]
print(len(badGroupsInActivity))

In [ ]:
#perfecto, aún veo un par de índices que puedo categorizar, puedo usar la misma función para ello

In [ ]:
listaConCategoriasFusionables =["Surf-skiing","Boating","Fishing for sharks"]
listaConCategoriasFusionables = [str(ele) for ele in listaConCategoriasFusionables]
activitySerie=activitySerie.replace(
    "Surf-skiing",
"Surf skiing"  
)
activitySerie=activitySerie.replace(
    ["Rowing","Boating"],
"Sailing"  
)
activitySerie=activitySerie.replace(
    'Fishing for sharks',
"Shark fishing"  
)

In [ ]:
pd.DataFrame(activitySerie.value_counts())

In [ ]:
#En este punto creo que activity está muy limpio, lo añado y borro variables que no vaya a usar
dfClean["activity"] = activitySerie
del activitySerie
del badGroupsInActivity
dfNotNull = dfNotNull.drop(columns = "activity")

In [ ]:
#Voy a ver si href y hrefFormula es información duplicada
import numpy as np
array = (dfNotNull["href"].values == dfNotNull["hrefFormula"].values)
array.size - np.sum(array)

In [ ]:
#Programé una función para comprobar si la url que se le pasa es válida y el archivo exite, si lo es devuelve la Url, si no Unknown
from functions import checkUrlValid


In [ ]:
#ok ya tengo las válidas de href, ahora solo voy a pasar la función en href formula para los índice unknown por si puedo salvar alguna
"""validHref = dfNotNull["href"].apply(checkUrlValid)
validHref"""

In [ ]:
"""voy a guardar el df en un csv, porque la función es lenta, creo que la razón es que el servidor
solo deja hacer unas pocas peticiones por unidad de tiempo"""

In [ ]:
"""validHref = validHref.astype(str)
pd.DataFrame(validHref).to_csv("./validHrefDf.csv")"""

In [ ]:
#Compruebo que se guardó, href válidas a 1-09-2019 01:11 AM
validHref = pd.read_csv("./outputignore/validHrefDf.csv")
validHref

In [ ]:
#voy a ver las que no sirven
validHref["href"].value_counts()

Solo hay 4 que no sirven asique creo que es bastante coherente añadir esta lista como limpia, y teniendo encuenta que solo 54 no coinciden con href formula creo que ésta última se puede dropear

In [ ]:
dfClean["href(verified1-09-2019)"] = validHref["href"]

In [ ]:
dfNotNull["country"].value_counts().head(70)

In [ ]:
#Pienso que country está bastande limpio así que lo voy a añadir tal cual
dfClean["country"] = dfNotNull["country"]
dfNotNull=dfNotNull.drop(columns="country")

In [ ]:
#Voy a analizar la columna de sex, parece facil
dfNotNull["sex"].value_counts().index.tolist()


In [ ]:
dfNotNull["sex"] = dfNotNull["sex"].replace(["M "],"M")
dfNotNull["sex"] = dfNotNull["sex"].replace([".","lli","N"],"unknown")
dfClean["sex"]= dfNotNull["sex"]
dfNotNull =dfNotNull.drop(columns = "sex") 


In [ ]:
dfClean["sex"].value_counts().index.tolist()

In [ ]:
dfNotNull["species"].value_counts().head(40)
##Observo que en esta columna hay bastantes datos de longitud por lo que los voy a intentar separar y colocar en una columna de longitud

In [ ]:
from functions import getLength

In [ ]:
dfDirtyLenght = pd.DataFrame(dfNotNull["species"].astype("str").apply(getLength))

In [ ]:
#Ahora renombrando la columna ya  tengo un dataframe con datos de longitudes
dfDirtyLenght.columns = ["lenght"]

In [ ]:
dfDirtyLenght["lenght"][1]

Dejo para después las longitudes, voy a limpiar las especies ahora que puedo prescindir de las longitudes

In [ ]:
pd.DataFrame(dfNotNull["species"].value_counts()).index.tolist()

In [ ]:
#Ok, voy a identificar una lista de especies y para pasarla como parámetro a mi función de limpieza

In [ ]:
listSpecies =['WHITE','TIGUER','BULL','WOBBEGONG','BLACKSTRIP','MAKO','BLUE','RAGGEDTOOTH','ZAMBESI','BRONZE WHALER','OCEANIC','SPINNER','HAMMERHEAD','BLACKTIP','SALMON','DUSKY','REEF','LEOPARD','LEMON','SEVENGILL','BLACKTIP','DOGFISH','SEVENGILL','THRESHER','GALAPAGOS',"NURSE"]
from functions import cleanShark
dfClean["species"]=dfNotNull["species"].apply(cleanShark,speciesList = listSpecies)


In [ ]:
dfClean["species"].value_counts()


OK, mi columna de species está limpia, la dropeo de dfNotNull

In [ ]:
dfNotNull = dfNotNull.drop(columns = "species")


In [ ]:
href = "href(verified1-09-2019)"
dfClean[href].values.tolist()


In [3]:
import requests

In [4]:
def checkUrlValid(text):
    """analiza si un string contiene una url válida (404 no valida) y la devuelve si lo es, si no devuelve UNKNOWN"""
    
    try:
        
        r = requests.get(text, allow_redirects=False)
        
        
        
        

     
    
        return r.headers
    except:
            return "EXCEP" 
    

In [13]:
a = checkUrlValid("http://sharkattackfile.net/spreadsheets/pdf_directory/2016.09.18.a-NS.pdf")
checkUrlValid("http://sharkattackfile.net/spreadsheets/pdf_directory/2014.12.23.R-Goblk.pdf")


False

In [ ]:
r = requests.get("http://sharkattackfile.net/spreadsheets/pdf_directory/2014.12.23.R-Goblk.pdf")

In [ ]:
r.status_code

In [ ]:
validHref["href"].values